### 参考文献

+ [How to Build a Pairs Trading Strategy on Quantopian?](https://www.quantopian.com/posts/how-to-build-a-pairs-trading-strategy-on-quantopian)
+ [CiNii 論文 -  共和分性に基づく最適ペアトレード](https://ci.nii.ac.jp/naid/120006224721/)


# ペアトレーディング

ペアトレーディングは，マーケットの暴騰や急落といった，急な変化に巻き込まれずに収益を上げていく方法として昔から使われてるストラテジーです

ペアトレーディング（以下ペアトレ）とは，2つの銘柄XとYの価格差(**スプレッド**)が平均的には一定の値で推移しているが，**一時的に大きくなった時に，時間が経過すれば元の水準に戻ることを期待**してポジションを取るストラテジーです．

**X-Y**の差はいつもある程度の範囲内であるのに，ある日スプレッドがいつもより大きくなった場合，**Xが大きく値上りしているかYが大きく値下がりしている**と考えられます．従って，その差がいつもの水準に戻る事を期待して，**Xを売りYを買う**ポジションを取ります．そして，スプレッドがいつもの水準に戻った時に，反対売買（Xを買い戻しYを売る）を行い利益を確定させます．

一般的に，株式価格自体はランダムウォーク（酔っぱらいのオジサン歩き＝どっちに動くか分からない）に従い予測することは出来ないと考えられます．

一方，価格のスプレッドは，一定値の周りで変動するという現象がしばしば観測されます．このような現象を**経済時系列における共和分性**といいます．

株式価格のペアが共和分する場合，スプレッドは，**mean reversion** つまり，平均回帰性をもつと考えられます．この性質を使ったストラテジーがペアトレです．


このノートでは，Quantopian Research を使ってどのようにペアトレを構築していくかを学んで行きます．ノート修了時には，

+ ペアトレ構築方法
+ 銘柄ペアの探し方
+ 銘柄ペアの評価方法
+ 共和分とは何か
+ どのように共和分をテストするか
+ どのように取引シグナルを作るか
+ python library pandas の機能の一部
+ matplotlib の機能の一部

を理解していることを目標にします．

## ペアトレーディングとは

ペアトレーディングとは，価格差がだいたい一定値に収まる2つの銘柄を探し，価格差がいつもの水準よりも大きくなった時に，通常の価格差に戻る事を期待して，一方を買い一方を売り，価格差が戻った時にポジションを閉じるストラテジーです．

ここでポイントになるのは**価格差がいつもだいたい一定値に収まる** というところです．

例を出します．

以下は金採掘会社関連銘柄GDXとABXの2013/1/3〜2013/9/3までの値動きとその差額，そして差額の平均をプロットしたものです．


In [ ]:
df = get_pricing(["GDX","ABX"], fields='price',start_date='2013-01-03', end_date='2013-09-03')
df['spread'] = df[symbols("GDX")] - df[symbols("ABX")] 
df['spread mean'] = df["spread"].mean()
df.plot()

spread が平均（約9.44）の周りを行ったり来たりしているのがわかります．

このように一定値の近傍（きんぼう）で変動する現象を**共和分性を持つ**と言います．

**価格差がいつもだいたい一定値に収まる＝共和分する**銘柄ペアを見つける事がペアトレの絶対条件になります．

では，なぜ共和分する銘柄ペアでペアトレをすると安定的に儲かるのか，考えてみます．


## 銘柄XとYで検証

ここでは，XとYという偽の銘柄の株価時系列（ヒストリカルデータ）を作り，共和分性と確かめ，実際の取引シュミュレーションを行います．

Data分析ライブラリのpandasや，描画ライブラリ matplotlib の使い方も一緒に学んで行きます．知らない人にとっては( ﾟдﾟ)!?となることもあると思いますので，遠慮せずお近くの優しいチューターさんに手を上げて聞いて下さい．

### XとYの作り方

XとYは同じような値動き（＝毎日同じような変化率で動く）をする銘柄でなくては行けません．

したがって，まずはXを作りそれをベースにYを作って行きます．

作り方は色々考えられますが，ココでは次の手順で作ってみます．

1. ランダムな数値を正規分布に従って100個作成
1. その数列をXの日々の変化率とし,その累積和をXの時系列データとする
1. もうひとつランダムな数列を作る
1. 時系列Xにこの数列を足して時系列Yとする


In [ ]:
import numpy as np
import pandas as pd

import statsmodels
from statsmodels.tsa.stattools import coint
np.random.seed(107)

import matplotlib.pyplot as plt

In [ ]:
X_returns = np.random.normal(0,1,100) #平均０，分散１のランダムな数値を100個作成
df = pd.DataFrame({"X":X_returns})
df["X"] = df["X"].cumsum()

some_noise = np.random.normal(0,1,100) 
df["Y"] = df["X"] + some_noise

df.head()


In [ ]:
df.plot()


いい感じですね．もう少し本物らしくするために全体に50を足し, Yには更に5を足しておきましょう．（株価がマイナスというのはありませんので）


In [ ]:
df = df+50
df["Y"] = df["Y"]+5


In [ ]:
df.plot()

### 共和分とは

X，Yが出来ました．

次にこの2つの銘柄を使って，**共和分**について考えていきます．

共和分とは，一定値の周りをウロウロするような現象のことでしたね．

XとYが共和分性があるかどうか，確かめてみましょう．



In [ ]:
df["spread"] = df["Y"] - df["X"]
df.head()

In [ ]:
ax = df["spread"].plot()
ax.axhline(df["spread"].mean(), color="red", linestyle="--")
ax.set_title("cointegration test")


spread（青線）が，その平均線（赤い点線）の周りと行ったり来たりしています．スプレッドが一定値の近傍で変動しているということで，このXとYは共和分性があることが確認できました．

次にこれを統計的に確認してみましょう．


### 共和分をテストする

python の統計モデルライブラリ [StatsModels](http://www.statsmodels.org/dev/index.html) の `statsmodels.tsa.stattools`の中にある [coint](http://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.coint.html) を使ってXとYが共和分しているかどうか確認します．

`coint(X,Y)` は３つの値 coint_t, pvalue, crit_value を返します．

そのなかの，pvalue が小さければ小さいほど，2つの価格は共和分性を持つことを示します．


In [ ]:
coint_t, pvalue, crit_value = coint(df["X"],df["Y"])
pvalue

### 取引シュミュレーション

では，私達のXとYでペアトレードをするとどうなるか．確認してみましょう．

上記のcointegration testの描画から推察するに，だいたいXとYの差の平均である**5**を挟んで行ったり来たりしています．

差は，**Y-X**で取得しましたので，平均より高い位置にあるということは，**Xがいつもより低すぎてYがいつもより高すぎる**
ということになります．

よって平均よりも**Spreadが上にある場合はXの買いYの売り，反対に下にある場合はXの売りYの買い**というストラテジーをとります，



In [ ]:
## ここの説明は pandas 博士のみなさんにお願いしたい．
df_test = df.copy()
df_test["above mean"] = df_test["spread"] > df_test["spread"].mean()
df_test["X_return"] = df_test["X"].pct_change().shift(-1)
df_test["Y_return"] = df_test["Y"].pct_change().shift(-1)
df_test["PL"] = (df_test["Y_return"] - df_test["X_return"]) * ~df_test["above mean"] \
              + (df_test["X_return"] - df_test["Y_return"]) * df_test["above mean"] 

In [ ]:
df_test["PL"].dropna().describe()

In [ ]:
df_test["PL"].cumsum().plot()

![](https://4.bp.blogspot.com/-zTvzECyWEsk/VwIjHWMdszI/AAAAAAAA5e4/W_kAnVythXoHGzGO3AkgrHImS3cpvMiuQ/s330/internet_kanki_man1.png)

まあ人生そんなに上手く行くことはありません．

でもこれで，共和分する銘柄ペアを探せばチャンスがあることがわかりました．

では，<font color=red>相場にある全銘柄でペアを作り，片っ端からその差の共和分を取得し，良いペアを探してみましょう!</font>
とやりたいところですが，さすがに大変なので，関連ありそうな銘柄を複数集めて，そこで総当りさせるコードを書いてみましょう．

### 関連のありそうな銘柄とは

一般的に同じような値動きをする銘柄は，同じ業種の銘柄や同じ商品のサプライヤー等，経済的に関連がありそうな銘柄が似たような値動きをします．（例：資生堂と花王（化粧品）や，村田製作所とアルプス電気（iphoneサプライヤ））

Quantopianでは，アメリカ株だけ取得可能ですので，ここでは金鉱株を集めて共和分を見ていきたいと思います．



In [ ]:
import itertools
def find_cointegrated_pairs(pairlist, start_date="2011-1-1", end_date="2018-1-1",):
    prices = get_pricing(pairlist, start_date=start_date, end_date=end_date, fields="price", frequency="daily")
    prices.columns =map(lambda x: x.symbol, prices.columns)
    pairs = list(itertools.combinations(pairlist, 2))

    pvalue_list = list()
    for a,b in pairs:
        _,pvalue,_ = coint(prices[a], prices[b])
        pvalue_list.append([a,b,pvalue])
        
    return sorted(pvalue_list, key=lambda x: x[2])  


In [ ]:
pairlist = ['AEM', 'GG', 'AUY', 'KGC', 'EGO', 'ABX', 'NEM', 'GDX']
find_cointegrated_pairs(pairlist)

もちろん，私達のXとYほどではありませんが，[ゴールドコープ【GG】](https://stocks.finance.yahoo.co.jp/us/detail/GG)と[エルドラド・ゴールド【EGO】](https://stocks.finance.yahoo.co.jp/us/detail/EGO)は悪くない感じですね.

では，この2つの銘柄を2011年から2017年末まで取得して，実際のシュミュレーションを行ってみましょう．


In [ ]:
prices = get_pricing(['GG', 'EGO'], start_date="2011-1-1", end_date="2017-12-31", fields="price", frequency="daily")
prices.columns =map(lambda x: x.symbol, prices.columns)
prices.plot()

In [ ]:
spread = prices["GG"]- prices["EGO"]
ax = spread.plot()
ax.axhline(spread.mean(), color="red", linestyle="--")


私達のXとYのようには行かないようですね．

では，**20日間のスプレッドの移動平均**を取って,それより上であればEGO買いGG売り，下であればEGO売りGG買いというストラテジーを組んでみましょう．

まずはプロットしてみます


In [ ]:
term = 20 
spread.name = "Spread"
ma = spread.rolling(term).mean()
ma.name = "{} mean".format(term)
ax = spread.plot()
ma.plot(ax=ax)
ax.legend()


Spreadが20meanより上にある場合は，GGを売りEGOを買い，逆の場合は逆ポジションを持つというストラテジーになります

In [ ]:
prices = get_pricing(['GG', 'EGO'], start_date="2008-1-1", end_date="2018-1-1", fields="price", frequency="daily")
prices.columns =map(lambda x: x.symbol, prices.columns)

prices["spread"] = prices["GG"]- prices["EGO"]
prices["spread MA {}".format(term)] = prices["spread"].rolling(term).mean()
prices["above mean"] = prices["spread"] > prices["spread MA {}".format(term)] 
prices["GG return"] = prices["GG"].pct_change().shift(-1)
prices["EGO return"] = prices["EGO"].pct_change().shift(-1)


In [ ]:
prices = prices.dropna()
prices["PL"] = (prices["EGO return"] - prices["GG return"]) * ~prices["above mean"] \
             + (prices["GG return"] - prices["EGO return"]) * prices["above mean"]
spy = get_pricing("SPY", start_date=prices.index[0], end_date=prices.index[-1], fields='price', frequency='daily')    

In [ ]:
prices["PL"].cumsum().plot()
spy.pct_change().cumsum().plot()


うーーーーん・・・2015年以降はまあまあいいのですが，変動が激しくツライですし，これではSPYを黙って持っている方が全然ましということになります．

と，なかなか人生うまく行かないわけですが，せっかくですので，みんなで手分けして良いストラテジーを探してみましょう．

思いついた改良ポイントを紹介しますので，これをヒントに改良してみてもらってもよいですし，何かご自身で思いつくモノがあればやってみて下さい．

(๑•̀ㅂ•́)و✧

### 改良（？）ヒント

+ 違うペアを探す
+ 違う業種を探す [S&P 500 Map](https://finviz.com/map.ashx?t=sec)
+ 複数ペアの組み合わせ
+ term をもっと短く/長く
+ 20日移動平均+ボリンジャーバンドの組み合わせ



## このあとは

アルゴリズムでシミュレーションテンプレートを作っておく？
